# Vigilancia epidemiológica

Cada mes, cientos de establecimientos de salud en Bolivia envían información sobre la prevalencia de enfermedades que el Estado considera de relevancia epidemiológica. Esta información es consolidada y presentada en [tableros](https://estadisticas.minsalud.gob.bo/Default_Vigilancia.aspx) que creo podrían ser mucho más amigables de usar. Hay tantos usos como visualizaciones, modelos de proyección o simples índices de incidencia, que en el estado actual serían demasiado tediosos de producir. Por eso, en este trabajo produzco una forma reproducible y escalable de descargar todos estos datos a nivel municipal y mensual para cada condición registrada y año entre 2001 y 2021.

In [1]:
# dependencias

import pandas as pd
import requests
import datetime as dt
from bs4 import BeautifulSoup
import unicodedata
from slugify import slugify
from IPython.display import clear_output
from itables import show, init_notebook_mode
init_notebook_mode()
from IPython.display import display, IFrame

<IPython.core.display.Javascript object>

Antes que nada, los datos. En este tablero puedes explorar algunas vistas interesantes:

In [4]:
display(IFrame(src='https://observablehq.com/embed/@mauforonda/vigilancia-epidemiologica?cells=barplot%2Ctext1%2Cviewof+year%2Cviewof+variable%2Cviewof+column%2Cviewof+minvalue%2Cheading1%2Clegend%2Cviewof+munselect%2Cheading2%2Clineplot%2Ctabla%2Ctext3%2Ctext2', width="100%", height="2035"))

Y en esta tabla están listados todos los datos disponibles, con enlaces que llevan a documentos en formato csv:

In [14]:
def draw_table(df):
    
    def link(dfi, name, link):
        return dfi[[link, name]].apply(lambda row: '<a href="{}">{}</a>'.format(row[0], row[1]), axis=1)
    
    dfi = df.copy()
    url_base = 'https://mauforonda.github.io/vigilancia-epidemiologica/datos/'
    dfi['filename'] = dfi.filename.apply(lambda f: '{}{}'.format(url_base, f))
    dfi['variable'] = link(dfi, name='variable', link='filename')
    dfi = dfi[['year', 'grupo', 'variable']]
    dfi.columns = ['Año', 'Grupo', 'Variable']
    
    show(dfi, 
         order = [],
         hover = True,
         compact=True,
         scrollY="900px", 
         lengthMenu=[50,100],
         scrollCollapse=True,
         search={"caseInsensitive": True},
         paging=True,
         language={
             'lengthMenu': 'Mostrar _MENU_ filas',
             'search': '&#x1F50E;&#xFE0E;', 
             'processing': 'creando tabla ...', 
             'info': '', 
             'infoEmpty': '', 
             'infoFiltered':'_TOTAL_ documentos',
             'paginate': {
                'first': "Primero",
                'previous': "Anterior",
                'next': "Siguiente",
                'last': "Último"
            },
         }, 
         maxBytes=0,
         columnDefs=[
             {"width": "5px", "targets": [0]},
             {"width": "20px", "targets": [1]},
             {"width": "100px", "targets": [2]}
         ]
        )

draw_table(indice)

Loading... (need help?)


A continuación detallo el proceso de construcción de estos datos, que consiste en 3 pasos:

1. Construir un inventario de variables
2. Descargar cada datos disponible
3. Preparar los datos para que sean utilizados fácilmente

## Un inventario de variables

*Qué información está disponible?*

El ministerio construye una página diferente para mostrar los datos de cada año. Es necesario consultar cada página y navegar los menús que ofrece para listar las variables y grupos de variables que exhibe. Algo interesante de estos tableros es que para entregar información, además de los parámetros que describen la consulta e identidad del usuario, requieren valores que describen el estado de navegación del usuario en el tablero durante la consulta anterior. Es decir que para mostrar una tabla en particular es necesario no sólo ejecutar la consulta correcta, sino la secuencia previa de consultas correctas que el servicio espera. Por suerte, estas limitaciones son fáciles de resolver.

In [88]:
# Información que el servidor espera de un usuario normal

cookies = {
    'ASP.NET_SessionId': 'es15y505g3h0e14ooob5ebns', # Place your own 🍪 🍪 🍪
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Origin': 'https://estadisticas.minsalud.gob.bo',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

In [310]:
# Some helper dicts

years = [
    {'year': 2001, 'pagina': 'Form_Vigi_2001.aspx'},
    {'year': 2002, 'pagina': 'Form_Vigi_2001.aspx'},
    {'year': 2003, 'pagina': 'Form_Vigi_2001.aspx'},
    {'year': 2004, 'pagina': 'Form_Vigi_2001.aspx'},
    {'year': 2005, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2006, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2007, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2008, 'pagina': 'Form_Vigi_2008.aspx'},
    {'year': 2009, 'pagina': 'Form_Vigi_2009.aspx'},
    {'year': 2010, 'pagina': 'Form_Vigi_2010.aspx'},
    {'year': 2011, 'pagina': 'Form_Vigi_2011.aspx'},
    {'year': 2012, 'pagina': 'Form_Vigi_2012.aspx'},
    {'year': 2013, 'pagina': 'Form_Vigi_2013.aspx'},
    {'year': 2014, 'pagina': 'Form_Vigi_2014_2.aspx'},
    {'year': 2015, 'pagina': 'Form_Vigi_2015_302a.aspx'},
    {'year': 2016, 'pagina': 'Form_Vigi_2016_302a.aspx'},
    {'year': 2017, 'pagina': 'Form_Vigi_2017_302a.aspx'},
    {'year': 2018, 'pagina': 'Form_Vigi_2018_302a.aspx'},
    {'year': 2019, 'pagina': 'Form_Vigi_2019_302a.aspx'},
    {'year': 2020, 'pagina': 'Form_Vigi_2020_302a.aspx'},
    {'year': 2021, 'pagina': 'Form_Vigi_2021_302a.aspx'}
]

state = {}

paginas = {y['year']:y['pagina'] for y in years}

In [360]:
# Functions to build a list of available datasets

def get_subvars(url, year, grvar):
    """
    Makes a list of every variable under a variable group and year.
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_grvar',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '0',
        'ctl00$MainContent$WebPanel2$List_grvar': grvar
    }

    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers=headers, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    subvars = [{'subvar': option['value'], 'variable': option.get_text().strip()} for option in html.select('#MainContent_WebPanel2_Lista_subvar option')]
    return subvars

def get_variables(year, pagina):
    """
    Makes a list of all variables and variable groups for a year. 
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_gestion',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$List_grvar': '01',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '0',
    }
    
    url = 'https://estadisticas.minsalud.gob.bo/Reportes_Vigilancia/{}'.format(pagina)
    variables[year] = []
    response = requests.get(url, cookies=cookies, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    response = requests.post(url, cookies=cookies, headers=headers, data={**data, **state})   
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    for option in html.select('#MainContent_WebPanel2_List_grvar option'):
        grupo = option.get_text().strip()
        grvar = option['value']
        subvars = get_subvars(url, year, grvar)
        for v in subvars:
            variables[year].append({'grvar': grvar, 
                                    'grupo': grupo,
                                    'subvar': v['subvar'],
                                    'variable': v['variable']})

def format_variables(variables):
    """
    Format variables and variable groups so that they're meaningful and easier to harmonize in the future.
    """

    dfv = []
    for y in variables.keys():
        dfi = pd.DataFrame(variables[y])
        dfi.insert(0, 'year', y)
        dfv.append(dfi)
    dfv = pd.concat(dfv)

    dfv.variable = dfv.variable.str.replace('^[0-9\.\-]+ ', '', regex=True)
    dfv.variable = dfv.variable.str.lower()
    dfv.variable = dfv.variable.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

    dfv.grupo = dfv.grupo.str.lower()
    dfv.grupo = dfv.grupo.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))
    
    return dfv.reset_index(drop=True)

In [321]:
# Make a list of all variables and variable groups for every year and format them right

variables = {}
for year in years:
    print(year['year'])
    get_variables(year['year'], year['pagina'])

dfv = format_variables(variables)

In [710]:
# save

dfv.to_csv('resources/variables.csv', index=False)

Encuentro más de 1500 variables que podría consultar entre 2001 y 2021. El inventario se ve así:

In [709]:
dfv

,year,grvar,grupo,subvar,variable
0,2001,00,clasificacion general sistemica,01,enfermedades del sistema nervioso
1,2001,00,clasificacion general sistemica,02,enfermedades del ojo y anexos
2,2001,00,clasificacion general sistemica,03,enfermedades del oido y apofisis mastoides
3,2001,00,clasificacion general sistemica,04,enfermedades del sistema cardio circulatorio
4,2001,00,clasificacion general sistemica,05,enfermedades del sistema respiratorio
...,...,...,...,...,...
1565,2021,16,eventos - notificacion inmediata parte ii,03,otros de excepcion
1566,2021,17,enfermedades transmitidas por vectores (etv),01,leishmaniasis
1567,2021,17,enfermedades transmitidas por vectores (etv),02,chagas agudo
1568,2021,17,enfermedades transmitidas por vectores (etv),04,malaria


En esta tabla, los valores `grvar` y `subvar` son códigos que describen un grupo de variables y variables en el tablero de un año `year`. Estos códigos serán útiles para solicitar datos al servidor en la próxima sección. Las condiciones, códigos y forma de los datos varían demasiado entre distintos años. Por eso, para tener un proceso que funcione a través de todas estas condiciones y años es importante sostener la menor cantidad de supuestos sobre cómo realizar consultas correctas o cómo se ve una tabla específica.

## Datos

*Descargar todos los datos disponibles a nivel mensual y municipal.*

Con el inventario en mano, el próximo paso es la descarga. Realizo consultas para cada variable, año y mes, que almaceno en documentos csv, uno para cada variable y año. Como no puedo predecir la forma que tendrá la tabla que produce el sistema, y éste tiende a nombrar columnas de maneras idiosincráticas, por ejemplo desagregando valores entre distintos rangos de edad, usando ocasionalmente sexo, etc., simplemente tomo todo lo que ofrece. Las únicas columnas sobre las que tengo control son las que creo, qu describen las variables, años y municipios. Las variables y años están bien harmonizadas gracias al índice. Y respecto al municipio, para tener datos que puedo cruzar fácilmente con otros sets de datos en la próxima sección construyo un diccionario que me permite harmonizar códigos para municipios a través de todos los años.

In [362]:
def get_state(url, year):
    """
    Inicializa parámetros de estado para persuadir al sistema a que nos entregue datos
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_gestion',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$List_grvar': '01',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '0',
    }
    
    response = requests.get(url, cookies=cookies, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
        
    response = requests.post(url, cookies=cookies, headers=headers, data={**data, **state})   
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
        
def update_state(url, year, grvar):
    """
    Actualiza parámetros de estado para persuadir al sistema a que nos entregue datos
    """

    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_grvar',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomMunicip',
        'ctl00$MainContent$WebPanel2$seleccion': '0',
        'ctl00$MainContent$WebPanel2$List_mes': '1',
        'ctl00$MainContent$WebPanel2$List_grvar': grvar
    }
    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers={**headers, **{'Referer': url}}, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']

def get_month(url, year, mes, grvar, subvar):
    """
    Descarga datos para un mes en una variable y año.
    """

    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': '',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomMunicip',
        'ctl00$MainContent$WebPanel2$seleccion': '0',
        'ctl00$MainContent$WebPanel2$Button1': ' Procesar',
        'ctl00$MainContent$WebPanel2$List_mes': str(mes),
        'MainContentxWebPanel3xmydatagrid': '',
        'MainContentxWebPanel3xmydatagrid2': '',
        'ctl00$MainContent$WebPanel2$List_grvar': grvar,
        'ctl00$MainContent$WebPanel2$Lista_subvar': subvar
    }
    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers={**headers, **{'Referer': url}}, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    table = html.select('#G_MainContentxWebPanel3xmydatagrid')[0]
    
    df = pd.read_html(str(table))[0]
    df = df.set_index(df.columns[0])
    df = df[~df.index.str.lower().str.contains('total')]
    
    return df
    
def get_dataset(year, grvar, subvar, grupo, variable):
    """
    Descarga datos para cada mes en una variable y año.
    """
    
    year_data = []
    url = 'https://estadisticas.minsalud.gob.bo/Reportes_Vigilancia/{}'.format(paginas[year])
    get_state(url, year)
    update_state(url, year, grvar)
    for mes in range(1,13):
        mes_data = get_month(url, year, mes, grvar, subvar)
        mes_data.insert(0, 'mes', dt.date(year, mes, 1))
        year_data.append(mes_data)
    year_data = pd.concat(year_data)
    year_data.insert(0, 'variable', variable)
    year_data.insert(0, 'grupo', grupo)
    filename = 'dirty/{}_{}_{}.csv'.format(year, slugify(grupo), slugify(variable))
    indice.append(dict(year=year, filename=filename, grvar=grvar, grupo=grupo, subvar=subvar, variable=variable))
    year_data.to_csv(filename)

In [377]:
indice = []

# Descargar datos para cada fila en el inventario y simultáneamente construir un índice

for i, row in dfv.iterrows():
    clear_output(wait=True)
    print('{}/{} : {} en {}'.format(i+1, len(dfv), row['variable'], row['year']))
    get_dataset(row['year'], row['grvar'], row['subvar'], row['grupo'], row['variable'])

indice = pd.DataFrame(indice)
indice.to_csv('snis/resources/indice.csv', index=False)

Además de seguir las reglas mínimas que el servicio demanda para acceder a datos, no hago nada fancy en estas consultas. Son consultas secuenciales, en parte porque no quiero arriesgar degradar la disponibilidad del sitio y también porque es el proceso más sencillo de programar y evaluar. En total debe haber tomado alrededor de 9 horas de consultas y, si bien el proceso es reproducible, espero que no sea algo que deba hacerse de manera rutinaria.

## Limpieza

*Que cada tabla tenga una forma predecible y municipios harmonizados.*

Finalmente, pongo los datos en una estructura simple y extensible. Las primeras columnas de todos los sets de datos describen el código INE del municipio, el nombre del municipio que ofrece el tablero, el grupo de variable, el nombre de la variable y el mes de los valores. Para tener los códigos INE, construyo un diccionario que mapea cada nombre de municipio con su código, utilizando las tablas que publica el ministerio de salud sobre la estructura de establecimientos entre 2005 y 2021. Por suerte, los nombres en estas tablas y la base de datos de vigilancia epidemiológica son las mismas, debido a que el reporte se suele realizar mediante un sistema que consume listas similares publicadas periódicamente. El resto de las columnas consisten en todos los datos que ofrece el sistema, a veces desagregados por rangos de edad, sexo, si la atención fue prestada en el establecimiento o fuera, etc. No hago más que la limpieza más simple sobre estas columnas, suficiente como para que sean utilizadas rápidamente en el futuro y sin muchos juicios sobre cómo deberían verse. Con esta información es fácil agregar valores de condiciones a través de varios años y cruzarlos con información a nivel municipal, como por ejemplo indicadores de los Objetivos de Desarrollo Sostenible.

In [685]:
def format_vigilancia(indice_seleccion, municipio_dict):
    
    errores = []
    len_seleccion = len(indice_seleccion)

    for i, f in enumerate(indice_seleccion.filename):
        clear_output(wait=True)
        print('{} / {}'.format(i+1, len_seleccion))
        try:

            dfi = pd.read_csv('dirty/{}'f)

            if dfi.mes.isna().sum() > 0:
                header = dfi[dfi.mes.isna()][dfi.columns[4:]].to_dict()
                header = ['_'.join([slugify(k)] + [slugify(header[k][kk]) for kk in header[k].keys() if type(header[k][kk]) == str ]) for k in header.keys()]
                dfi = dfi[dfi.mes.notna()]
                dfi.columns = ['municipio', 'grupo', 'variable', 'mes'] + header

            else:
                dfi = dfi.rename(columns={'Municipio':'municipio'})

            dfi.insert(0, 'codigo_municipio', dfi.municipio.map(municipio_dict))
            dfi.to_csv('datos/{}'.format(f.split('/')[1]), index=False)

        except Exception as e:
            errores.append({'filename': f, 'error': e})

    return errores

def listar_municipios1(f):
    
    municipios_snis = []

    establecimientos = pd.ExcelFile(f)

    for s in establecimientos.sheet_names:
        e = pd.read_excel(establecimientos, sheet_name=s, header=2)
        e = e[['COD_MUNICIPIO', 'MUNICIPIO']]
        e = e.drop_duplicates()
        municipios_snis.append(e)

    municipios_snis = pd.concat(municipios_snis)
    municipios_snis = municipios_snis[municipios_snis.COD_MUNICIPIO.notna()]
    municipios_snis['COD_MUNICIPIO'] = municipios_snis.COD_MUNICIPIO.astype(int)
    municipios_snis['MUNICIPIO'] = municipios_snis.MUNICIPIO.str.strip()
    municipios_snis = municipios_snis.drop_duplicates()
    
    return municipios_snis

def listar_municipios2(f):
    e = pd.read_excel(f, sheet_name='BASE DE DATOS', header=3)
    e = e[['COD_MUN', 'MUN']]
    e.columns = ['COD_MUNICIPIO', 'MUNICIPIO']
    e = e[e.MUNICIPIO.notna()]
    e['COD_MUNICIPIO'] = e.COD_MUNICIPIO.astype(int)
    e['MUNICIPIO'] = e.MUNICIPIO.str.strip()
    e = e.drop_duplicates()
    return e

In [691]:
# Construir un diccionario para harmonizar municipios

municipios_snis = pd.concat([
    listar_municipios1('snis/resources/Establecimientos 2005_2017.xlsx'), 
    listar_municipios2('snis/resources/ESTRUCTURA DE EE.SS. GESTION 2021_DASHBOARDcerrado oficial.xlsx')
]).drop_duplicates()

municipios_snis['MUNICIPIO'] = municipios_snis.MUNICIPIO.apply(lambda x: ' '.join(x.split()))
municipios_dict = municipios_snis.set_index('MUNICIPIO').COD_MUNICIPIO.to_dict()

In [692]:
errores = format_vigilancia(indice, municipios_dict)

1570 / 1570


Para terminar, así es como se ve uno de los más 1500 sets de datos escogido aleatoriamente:

In [9]:
pd.read_csv('datos/{}'.format(indice.sample()['filename'].iloc[0]))

,codigo_municipio,municipio,grupo,variable,mes,< de 1 año-H,< de 1 año-M,1 a 4 años-H,1 a 4 años-M,5 a 14 años-H,5 a 14 años-M,15 a 59 años-H,15 a 59 años-M,60 y más-H,60 y más-M,TOTAL-H,TOTAL-M,TOTAL
0,51302,ACACIO,clasificacion general sistemica,otras causas,2002-01-01,1,0,0,0,0,0,5,0,2,0,8,0,8
1,20201,ACHACACHI,clasificacion general sistemica,otras causas,2002-01-01,1,0,3,0,5,0,27,0,8,0,44,0,44
2,20104,ACHOCALLA,clasificacion general sistemica,otras causas,2002-01-01,1,0,3,0,6,0,10,0,6,0,26,0,26
3,30201,AIQUILE,clasificacion general sistemica,otras causas,2002-01-01,18,0,13,0,10,0,66,0,8,0,115,0,115
4,31303,ALALAY,clasificacion general sistemica,otras causas,2002-01-01,0,0,1,0,0,0,5,0,1,0,7,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,31201,TOTORA(CBBA),clasificacion general sistemica,otras causas,2002-12-01,0,0,0,0,1,0,18,0,1,0,20,0,20
1936,41301,TOTORA(ORR),clasificacion general sistemica,otras causas,2002-12-01,0,0,1,0,5,0,1,0,0,0,7,0,7
1937,31302,VILA VILA,clasificacion general sistemica,otras causas,2002-12-01,1,0,1,0,0,0,2,0,1,0,5,0,5
1938,10801,VILLA SERRANO,clasificacion general sistemica,otras causas,2002-12-01,4,0,23,0,11,0,41,0,4,0,83,0,83


Puedes volver a la tabla interactiva al inicio, descargar un csv y jugar con él en el sistema con el que te sientas más cómodo.